In [ ]:
import pandas as pd
import numpy as np
import pygsheets
#authorization
gc = pygsheets.authorize(service_file='viz-midterm-56fb8f946d76.json')
sh = gc.open('viz midterm data')

missing_countries_list = []


In [ ]:
# parse military data

xldata = pd.ExcelFile("https://www.sipri.org/sites/default/files/SIPRI-Milex-data-1949-2017.xlsx")

# Data for military expenditure by country in current US$ (millions), presented according to calendar year.
me_total_usd_main = pd.read_excel(xldata,sheet_name="Current USD",skiprows=5)
# Data for military expenditure by country as a share of GDP, presented according to calendar year.
me_share_gdp_main = pd.read_excel(xldata,sheet_name="Share of GDP",skiprows=5)
# Data for military expenditure per capita, in current US$, presented according to calender year. (1988-2017 only)
me_percap_main = pd.read_excel(xldata,sheet_name="Per capita",skiprows=6)

#me_percap["Country"].loc(["USA","USSR"])
country_list = ["Argentina", "Australia", "Brazil", "Canada", "China", 
                "France", "Germany", "India", "Indonesia", "Italy", "Japan", 
                "Republic of Korea", "Mexico", "Russia", "Saudi Arabia", 
                "South Africa", "Turkey", "UK", "USA"]
#country_list = ["USA", "China", "Russia", "Germany", "UK", "France", "Italy", "Iran", "Israel", "Saudi Arabia"]
year_list = [2010,2011,2012,2013,2014,2015,2016,2017]


# this standardizes the naming conventions for russia and china, which are different between three data sets
me_total_usd_main.loc[me_total_usd_main["Country"].str.contains("Russia",na=False),"Country"] = "Russia"
me_share_gdp_main.loc[me_share_gdp_main["Country"].str.contains("Russia",na=False),"Country"] = "Russia"
me_percap_main.loc[me_percap_main["Country"].str.contains("Russia",na=False),"Country"] = "Russia"
me_total_usd_main.loc[me_total_usd_main["Country"].str.contains("China",na=False),"Country"] = "China"
me_share_gdp_main.loc[me_share_gdp_main["Country"].str.contains("China",na=False),"Country"] = "China"
me_percap_main.loc[me_percap_main["Country"].str.contains("China",na=False),"Country"] = "China"

# pick out year columns from 2010-2017 for relevant countries
me_total_usd = me_total_usd_main[me_total_usd_main["Country"].isin(country_list)][["Country"] + year_list]
me_share_gdp = me_share_gdp_main[me_share_gdp_main["Country"].isin(country_list)][["Country"] + year_list]
me_percap = me_percap_main[me_percap_main["Country"].isin(country_list)][["Country"] + year_list]

# melt to create "year" and "total_ME" columns
me_total_usd = me_total_usd.melt(id_vars="Country",var_name="year" ,value_name="total_ME")
me_share_gdp = me_share_gdp.melt(id_vars="Country",var_name="year" ,value_name="share_of_gdp_ME")
me_percap = me_percap.melt(id_vars="Country",var_name="year" ,value_name="per_capita_ME")

# merge all into one on country and year
me_df = pd.merge(pd.merge(me_total_usd,me_share_gdp, on=["Country","year"]), me_percap, on=["Country","year"])

#calculate gdp, population, per capita gdp, and share_of_gdp_ME
me_df["gdp"] = me_df["total_ME"]/me_df["share_of_gdp_ME"]
me_df["gdp"] = me_df["gdp"].astype(int)
me_df["pop"] = me_df["total_ME"]/me_df["per_capita_ME"]
me_df["pcgdp"] = me_df["gdp"]/me_df["pop"]
me_df["year"] = me_df["year"].astype(str)
me_df["share_of_gdp_ME"] = me_df["share_of_gdp_ME"]*100

print(len(me_df.Country.unique()) == len(country_list), ": ed_df names and country_list are same not length")
print()
print("Here are our G20 Countries: ",me_df.Country.unique())
print()
print("There are/is", len(list(set(country_list).difference([i for i in me_df.Country.unique()]))), "Countries missing")
print()
print(list(set(country_list).difference([i for i in me_df.Country.unique()])),"are/is missing")
missing_countries_list.extend(list(set(country_list).difference([i for i in me_df.Country.unique()])))


In [ ]:
#EDUCATION DATA SOURCE: https://data.oecd.org/eduresource/public-spending-on-education.htm
# COUNTRY CODES DATA SOURCE: https://unstats.un.org/unsd/tradekb/Knowledgebase/50347/Country-Code

# read in education funding data- EDUPUBEXP (public expenditure) in PC_GDP (percent gdp)
ed_data = pd.read_csv("data/education.csv")

# read in country codes and split country code from name into new columns
country_codes = pd.read_csv("data/country_codes.csv",header=None)
country_codes["LOCATION"], country_codes["name"] = country_codes[0].str.split(' ',1).str

# merge ed_df and country_codes to get  full country names from codes
ed_df_names = pd.merge(ed_data,country_codes,on='LOCATION')

# strip extra string from end of country names
ed_df_names['name'] = ed_df_names['name'].str.strip("\xa0")

# rename countries to match original country list (military data)
ed_df_names['name'] = ed_df_names['name'].replace({'United States': 'USA', 'Russian Federation': 'Russia', 'United Kingdom': 'UK', 'Korea, Republic of': 'Republic of Korea'})

#extract only rows of countries in country list and look at unique countries
ed_df_clean = ed_df_names[ed_df_names['name'].isin(country_list)]
print(len(ed_df_clean.name.unique()) == len(country_list), ": ed_df names and country_list are not same length")
print()
print("Here are our G20 Countries: ",ed_df_clean.name.unique())
print()
print("There are", len(list(set(country_list).difference([i for i in ed_df_clean["name"].unique()]))), "Countries missing")
print()
print(list(set(country_list).difference([i for i in ed_df_clean.name.unique()])),"are/is missing")
missing_countries_list.extend(list(set(country_list).difference([i for i in ed_df_clean.name.unique()])))


In [ ]:
# pull out rows with "PRY_NTRY" (primary not tertiary) for subject and years in our year list
ed_df_clean = ed_df_clean[(ed_df_clean["SUBJECT"] == 'PRY_NTRY') & (ed_df_clean["TIME"].isin(year_list))]
ed_df_clean.shape

In [ ]:
important_ed_cols = ["name","TIME","Value"]
ed_df = ed_df_clean[important_ed_cols]
ed_df.columns = ["Country","year","pct_gdp_ed"]
ed_df['year'] = ed_df['year'].astype(str)
ed_df.head()

In [ ]:
#HEALTHCARE DATA SOURCE: https://data.oecd.org/healthres/health-spending.htm
#read in healthcare data
hc_data = pd.read_csv('data/healthcare.csv')

#merge healthcare data with country_codes to get full country names
hc_df_names = pd.merge(hc_data,country_codes,on='LOCATION')

# strip extra string from end of country names
hc_df_names['name'] = hc_df_names['name'].str.strip("\xa0")

# rename countries to match original country list (military data)
hc_df_names['name'] = hc_df_names['name'].replace({'United States': 'USA', 'Russian Federation': 'Russia', 'United Kingdom': 'UK', 'Korea, Republic of': 'Republic of Korea'})

#extract only rows of countries in country list and look at unique countries
hc_df_clean = hc_df_names[hc_df_names['name'].isin(country_list)]
print(len(hc_df_clean.name.unique()) == len(country_list), ": ed_df names and country_list are not same length")
print()
print("Here are our G20 Countries: ",hc_df_clean.name.unique())
print()
print("There are", len(set(country_list).difference([i for i in hc_df_clean["name"].unique()])), 
      "Countries missing from healthcare data:",set(country_list).difference([i for i in hc_df_clean["name"].unique()]))
print()
print(list(set(country_list).difference([i for i in hc_df_clean.name.unique()])),"are/is missing")
missing_countries_list.extend(list(set(country_list).difference([i for i in hc_df_clean.name.unique()])))



In [ ]:
# pull out rows with "PRY_NTRY" (primary not tertiary) for subject and years in our year list
hc_df_clean = hc_df_clean[(hc_df_clean["SUBJECT"] == 'TOT') 
                          & (hc_df_clean["TIME"].isin(year_list)) 
                          & (hc_df_clean["MEASURE"] == 'PC_GDP')]
print(hc_df_clean.shape)
hc_df_clean.head()

In [ ]:
important_hc_cols = ["name","TIME","Value"]
hc_df = hc_df_clean[important_hc_cols]
hc_df.columns = ["Country","year","pct_gdp_hc"]
hc_df['year'] = hc_df['year'].astype(str)
hc_df.head()

In [ ]:
# Now merge all of the data together to make for easy transfer to google sheets!!!!


total_df = pd.merge(pd.merge(me_df, ed_df, on=['year','Country'], how='outer'), hc_df, on=['year','Country'], how='outer')
total_df = total_df.fillna(0)
# wks = sh[0]
# wks.title= 'all_data'
# wks.set_dataframe(total_df,(1,1))
# total_df.head()

In [ ]:
total_df["total_ED"] = (total_df["pct_gdp_ed"] * (total_df["gdp"]))/100
total_df["total_ED"] = total_df["total_ED"].astype(int)
total_df["per_capita_ED"] = (total_df["total_ED"]/(total_df["pop"]))
total_df["total_HC"] = (total_df["pct_gdp_hc"] * (total_df["gdp"]))/100
total_df["total_HC"] = total_df["total_HC"].astype(int)
total_df["per_capita_HC"] = (total_df["total_HC"]/(total_df["pop"]))

total_df = total_df.fillna(0)
total_df = total_df.replace(0,"null")
total_df.head()

In [ ]:
total_df_pivot = total_df.melt(id_vars=["year","Country"],value_vars=[
    "total_ME","share_of_gdp_ME","per_capita_ME","gdp","pop","pcgdp","pct_gdp_ed",
    "pct_gdp_hc","total_ED", "per_capita_ED", "total_HC", "per_capita_HC"
    ]).set_index(["variable","Country"]).pivot(columns='year').reset_index().set_index("Country").reset_index()
total_df_pivot.columns = ["Country","variable","2010","2011","2012","2013","2014","2015","2016","2017"]
total_df_pivot.head()

In [ ]:
#total Military Expenditure
total_me_df = total_df_pivot[
    total_df_pivot.variable == "total_ME"].drop(columns="variable").reset_index().drop(columns="index")

# drop korea because no data
total_me_df = total_me_df[total_me_df.Country != "Republic of Korea"]

# wks = sh[0]
# wks.title = "total_ME"
# wks.set_dataframe(total_me_df,(1,1))
total_me_df.head()

In [ ]:
total_ed_df = total_df_pivot[
    total_df_pivot.variable == "total_ED"].drop(columns="variable").reset_index().drop(columns="index")

# drop countries with no data
total_ed_df = total_ed_df[(total_ed_df.Country != "China") & 
                          (total_ed_df.Country != "Saudi Arabia") &
                         (total_ed_df.Country != "Republic of Korea")]
#drop columns with no values
total_ed_df = total_ed_df.drop(columns=["2016","2017"])

# sh.add_worksheet("total_ED")
# wks = sh.worksheet_by_title("total_ED")
# wks.set_dataframe(total_ed_df,(1,1))
total_ed_df.head()

In [ ]:
total_hc_df = total_df_pivot[
    total_df_pivot.variable == "total_HC"].drop(columns="variable").reset_index().drop(columns="index")

# remove countries with missing data
total_hc_df = total_hc_df[(total_hc_df.Country != "Argentina") &
                         (total_hc_df.Country != "Republic of Korea") &
                          (total_hc_df.Country != "Saudi Arabia")]
                          
# sh.add_worksheet("total_HC")
# wks = sh.worksheet_by_title("total_HC")
# wks.set_dataframe(total_hc_df,(1,1))
total_hc_df.head()

In [ ]:
#percent gdp Military Expenditure
pct_gdp_me_df = total_df_pivot[
    total_df_pivot.variable == "share_of_gdp_ME"].drop(columns="variable").reset_index().drop(columns="index")

# drop korea because no data
pct_gdp_me_df = pct_gdp_me_df[pct_gdp_me_df.Country != "Republic of Korea"]

# sh.add_worksheet("pct_gdp_ME")
# wks = sh.worksheet_by_title("pct_gdp_ME")
# wks.set_dataframe(pct_gdp_me_df,(1,1))
pct_gdp_me_df.head()

In [ ]:
pct_gdp_ed_df = total_df_pivot[
    total_df_pivot.variable == "pct_gdp_ed"].drop(columns="variable").reset_index().drop(columns="index")

# drop years with no data
pct_gdp_ed_df = pct_gdp_ed_df.drop(columns=["2016","2017"])
# drop countries with no data
pct_gdp_ed_df = pct_gdp_ed_df[(pct_gdp_ed_df.Country != "China") & 
                              (pct_gdp_ed_df.Country != "Saudi Arabia") &
                              (pct_gdp_ed_df.Country != "Republic of Korea")]

# sh.add_worksheet("pct_gdp_ED")
# wks = sh.worksheet_by_title("pct_gdp_ED")
# wks.set_dataframe(pct_gdp_ed_df,(1,1))
pct_gdp_ed_df.head()

In [ ]:
pct_gdp_hc_df = total_df_pivot[
    total_df_pivot.variable == "pct_gdp_hc"].drop(columns="variable").reset_index().drop(columns="index")

# remove countries with missing data
pct_gdp_hc_df = pct_gdp_hc_df[(pct_gdp_hc_df.Country != "Argentina") &
                         (pct_gdp_hc_df.Country != "Republic of Korea") &
                          (pct_gdp_hc_df.Country != "Saudi Arabia")]

# sh.add_worksheet("pct_gdp_HC")
# wks = sh.worksheet_by_title("pct_gdp_HC")
# wks.set_dataframe(pct_gdp_hc_df,(1,1))
pct_gdp_hc_df.head()

In [ ]:
total_df_pivot['variable'].unique()

In [ ]:
# #add a new sheet for every country
# for country in country_list:
#     sh.add_worksheet(country)

# #add country-specific data to new sheets
# for country in country_list:
#     wks = sh.worksheet_by_title(country)
#     new_frame = total_df[total_df["Country"]==country]
#     wks.set_dataframe(new_frame,(1,1))

In [ ]:
# some useful sheets api code
# Loop through each variable and create a new sheet of that variable per year per country in google sheets
# plot_vars = [""]
# for var in total_df_pivot['variable'].unique():
#     df = total_df_pivot[total_df_pivot.variable == var].drop(columns="variable").reset_index().drop(columns="index")
#     sh.add_worksheet(var)
#     wks = sh.worksheet_by_title(var)
#     wks.set_dataframe(df,(1,1))
#     print(var)
    
#delete worksheets when necessary
# for i in range(6):
#     sh.del_worksheet(sh[i])